<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Machine Learning Para Aplicações Biomédicas</font>
## <font color='blue'>Projeto 8</font>
### <font color='blue'>Analisando a Efetividade de Medicamentos em Medicina Personalizada com Algoritmos Genéticos</font>

## Pacotes Python Usados no Projeto

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

O procedimento de instalação de pacotes e dependências está no arquivo LEIAME.txt

https://deap.readthedocs.io/en/master/

In [2]:
#!pip install -q deap

Nota: O pacote deap ainda não tem versão para computadores da Apple com processador Apple Silicon (M1/M2/M3/M4). Uma alternativa é executar este projeto no Google Colab.

In [3]:
# Imports
import random
import numpy as np
from deap import base, creator, tools, algorithms

In [4]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Definindo Pacientes, Características e Medicamentos

In [5]:
# Definir número de pacientes, características e medicamentos
N_PATIENTS = 50
N_FEATURES = 10
N_MEDICATIONS = 10

In [6]:
# Gerar características aleatórias para os pacientes (entre 0 e 1)
patient_features = np.random.rand(N_PATIENTS, N_FEATURES)

In [7]:
# Gerar perfis de efetividade aleatórios para os medicamentos
medication_profiles = np.random.rand(N_MEDICATIONS, N_FEATURES)

In [8]:
# Função para calcular a efetividade de um medicamento em um paciente
def dsa_calcula_efetividade(patient, medication):
    
    # A efetividade é calculada como o produto escalar entre as características do paciente 
    # e o perfil do medicamento
    return np.dot(patient, medication)

## Configuração do DEAP Para o Algoritmo Genético
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
DEAP (Distributed Evolutionary Algorithms in Python) é utilizado para configurar e executar um algoritmo genético. A DEAP fornece classes e métodos que facilitam a implementação de algoritmos genéticos e outros algoritmos evolutivos.

In [9]:
# Classe para avaliar a qualidade de uma solução
creator.create("FitnessMax", base.Fitness, weights = (1.0,)) 

**creator.create**: A função create do módulo creator da DEAP é usada para criar novas classes. Neste caso, estamos criando uma classe chamada "FitnessMax", que será derivada da classe base.Fitness.

**base.Fitness**: A classe Fitness é fornecida pela DEAP para avaliar a qualidade de uma solução (ou "indivíduo") em um algoritmo genético. base.Fitness é a classe base que é personalizada com o argumento weights.
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
**weights = (1.0,)**: O argumento weights indica o tipo de otimização que será feito na função de fitness. Aqui, (1.0,) especifica que o algoritmo deve maximizar a função de fitness (ou seja, buscamos o maior valor possível de efetividade). Um valor positivo (1.0) indica maximização, enquanto um valor negativo indicaria minimização. Esse valor é importante para orientar o algoritmo na direção correta.

In [10]:
# Cada Individual terá um atributo fitness
creator.create("Individual", list, fitness = creator.FitnessMax)

**creator.create**: Novamente, a função create é usada para criar uma nova classe chamada "Individual".

**list**: A classe Individual será derivada de uma lista Python nativa. Isso significa que cada "Indivíduo" será essencialmente uma lista, onde cada elemento representa uma característica ou gene. No contexto do projeto, cada item da lista pode representar a atribuição de um medicamento específico para um paciente.

**fitness = creator.FitnessMax**: Este atributo define que cada Individual terá um atributo fitness que usa a classe FitnessMax. Esse atributo é essencial para avaliar o quão boa é a solução representada pelo indivíduo.

As duas linhas acima configuram a estrutura básica para os indivíduos e a função de fitness no algoritmo genético. A configuração estabelece que:
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
- Cada indivíduo é uma lista de atributos ou "genes", que representa uma possível solução para o problema de atribuição de medicamentos.
- O objetivo é maximizar a função de fitness, que neste caso representa a eficácia total do tratamento para os pacientes, levando em consideração as características individuais e os perfis dos medicamentos.

Essa configuração é a base para que o algoritmo evolua em direção à melhor solução de maneira iterativa, ao combinar e mutar indivíduos ao longo de várias gerações.

In [11]:
# Cria a toolbox
toolbox = base.Toolbox()

In [12]:
# Gerador de atributos: atribui um medicamento aleatório a cada paciente
toolbox.register("attr_medication", random.randint, 0, N_MEDICATIONS - 1)

In [13]:
# Inicializadores de estrutura
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_medication, n = N_PATIENTS)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

## Função Para Avaliar a Efetividade

A função dsa_avalia_efetividade() calcula a eficácia total de uma solução específica, que é representada por uma lista de medicamentos atribuídos a pacientes. A eficácia de cada atribuição é somada para obter o total, que é usado como critério de fitness pelo algoritmo genético para avaliar o quão boa é a solução em maximizar a efetividade do tratamento.
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

In [14]:
# Função de avaliação que calcula a efetividade total de uma atribuição de medicamentos
def dsa_avalia_efetividade(individual):
    
    # Inicializa a variável para armazenar a efetividade total
    total_effectiveness = 0

    # Itera sobre cada paciente e a atribuição de medicamento correspondente no indivíduo
    for patient_idx, medication_idx in enumerate(individual):
        
        # Seleciona o perfil do paciente com base no índice atual
        patient = patient_features[patient_idx]
        
        # Seleciona o perfil do medicamento com base no índice atribuído
        medication = medication_profiles[medication_idx]
        
        # Calcula a efetividade do medicamento para o paciente usando uma função específica
        effectiveness = dsa_calcula_efetividade(patient, medication)
        
        # Adiciona a efetividade calculada ao total de efetividade
        total_effectiveness += effectiveness

    # Retorna a efetividade total como uma tupla (exigido pelo DEAP para o fitness)
    return (total_effectiveness,)

## Define os Operadores Genéticos

Essas linhas configuram os operadores genéticos essenciais para o algoritmo genético utilizando o toolbox do framework DEAP. O toolbox é uma ferramenta central no DEAP que registra funções que definem a lógica do algoritmo genético, como avaliação de fitness, crossover, mutação e seleção. 

In [15]:
# Registro da função de avaliação de fitness
toolbox.register("evaluate", dsa_avalia_efetividade)

**toolbox.register("evaluate", dsa_avalia_efetividade)**: Aqui, a função de avaliação dsa_avalia_efetividade é registrada com o nome "evaluate". Esta função será chamada para calcular a efetividade total (fitness) de cada indivíduo na população.
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
**Função de Fitness**: dsa_avalia_efetividade avalia o quão boa é uma solução (ou indivíduo) em maximizar a efetividade do tratamento com medicamentos. No contexto do DEAP, cada vez que o algoritmo precisar avaliar uma solução, ele chamará a função registrada como "evaluate".

In [16]:
# Registro do operador de crossover (cruzamento) uniforme
toolbox.register("mate", tools.cxUniform, indpb = 0.2)

**toolbox.register("mate", tools.cxUniform, indpb=0.2)**: Aqui, a função tools.cxUniform é registrada como o operador de crossover, com o nome "mate". O cxUniform é um tipo de crossover uniforme que decide aleatoriamente para cada gene se ele será trocado entre dois indivíduos (pais).

**indpb=0.2**: Esse parâmetro define a probabilidade individual (indpb) de cada gene ser trocado entre os pais. Com indpb=0.2, cada gene tem 20% de chance de ser trocado. Crossover uniforme é útil para criar maior variação nos filhos ao combinar características dos pais.

In [17]:
# Registro do operador de mutação
toolbox.register("mutate", tools.mutUniformInt, low = 0, up = N_MEDICATIONS - 1, indpb = 0.05)

**toolbox.register("mutate", tools.mutUniformInt, low=0, up=N_MEDICATIONS - 1, indpb=0.05)**: Essa linha registra tools.mutUniformInt como o operador de mutação com o nome "mutate". O mutUniformInt é um operador de mutação que altera o valor de genes específicos com base em um intervalo definido.

**low=0 e up=N_MEDICATIONS - 1**: Esses parâmetros especificam que o valor de cada gene mutado será alterado para um valor aleatório entre 0 e N_MEDICATIONS - 1, onde N_MEDICATIONS representa o número de medicamentos disponíveis. Portanto, cada gene (representando a atribuição de um medicamento) pode ser modificado dentro desse intervalo.

**indpb=0.05**: Esta é a probabilidade de mutação para cada gene. Com indpb=0.05, cada gene tem 5% de chance de sofrer mutação. A mutação ajuda a introduzir variação e evita que o algoritmo fique preso em soluções subótimas.

In [18]:
# Registro do operador de seleção
toolbox.register("select", tools.selTournament, tournsize = 3)

**toolbox.register("select", tools.selTournament, tournsize=3)**: Aqui, o operador de seleção tools.selTournament é registrado com o nome "select". O selTournament é um método de seleção por torneio, onde grupos de indivíduos são selecionados aleatoriamente, e o mais apto dentro de cada grupo é escolhido para reprodução.
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
**tournsize=3**: O parâmetro tournsize define o tamanho do torneio, ou seja, quantos indivíduos competem entre si. Neste caso, três indivíduos são comparados a cada vez, e o que possui maior fitness é selecionado. A seleção por torneio é útil para balancear exploração e exploração, mantendo diversidade na população enquanto seleciona indivíduos mais aptos.

## Função Principal

Essa função encapsula o processo de evolução genética, ajustando a população através de várias gerações até chegar à melhor atribuição de medicamentos com base na maximização da efetividade do tratamento.

In [19]:
# Função principal
def main():
    
    # Para reproduzir os mesmos resultados
    random.seed(42)
    
    # População inicial de 100 indivíduos
    pop = toolbox.population(n = 100)  
    
    # Número de gerações
    NGEN = 50  
    
    # Probabilidades de crossover e mutação
    CXPB, MUTPB = 0.5, 0.2  

    # Avalia a população inicial
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # Loop das gerações
    for gen in range(NGEN):
        
        # Seleciona os indivíduos para a próxima geração
        offspring = toolbox.select(pop, len(pop))
        offspring = list(map(toolbox.clone, offspring))

        # Aplicar crossover e mutação nos indivíduos selecionados
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Avalia os indivíduos com fitness inválido
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Substitui a população antiga pela nova
        pop[:] = offspring

        # Estatísticas da geração atual
        fits = [ind.fitness.values[0] for ind in pop]
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("Geração %s: Máx %s, Média %s, Desvio Padrão %s" % (gen, max(fits), mean, std))

    # Seleciona o melhor indivíduo
    best_ind = tools.selBest(pop, 1)[0]
    print("Melhor indivíduo: %s, Fitness: %s" % (best_ind, best_ind.fitness.values))
    
    return best_ind

## Execução do Projeto

In [20]:
if __name__ == "__main__":
    
    # Executa a função principal
    best_individual = main()
    
    print("\n___\n")
        
    # Exibe a melhor atribuição de medicamentos para os pacientes
    print("\nMelhor atribuição de medicamentos para os pacientes:\n")
    for patient_idx, medication_idx in enumerate(best_individual):        
        print("O medicamento %d é o mais efetivo para o paciente %d" % (medication_idx, patient_idx))

Geração 0: Máx 123.9117990710523, Média 118.7084833032484, Desvio Padrão 1.8638742161144515
Geração 1: Máx 124.08614487544939, Média 120.43886700500349, Desvio Padrão 1.9260783079475408
Geração 2: Máx 125.7697089343586, Média 122.26115448586108, Desvio Padrão 1.4945639809740325
Geração 3: Máx 126.55955988571552, Média 123.41769773934026, Desvio Padrão 1.4222186345974388
Geração 4: Máx 128.7775568784764, Média 124.50054961856274, Desvio Padrão 1.5349221019978863
Geração 5: Máx 129.21014743582805, Média 125.80408596633391, Desvio Padrão 1.5620359370832282
Geração 6: Máx 129.7239443930908, Média 127.01015815801544, Desvio Padrão 1.3806253977160856
Geração 7: Máx 133.0564790988313, Média 128.01791804059286, Desvio Padrão 1.3472072581130818
Geração 8: Máx 133.0564790988313, Média 129.05905000931642, Desvio Padrão 1.5407477401019003
Geração 9: Máx 133.48486078911003, Média 130.10570887896523, Desvio Padrão 1.3299282316977903
Geração 10: Máx 133.88050056270194, Média 131.09846862967635, Desvi

In [21]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [22]:
#%watermark -v -m

In [23]:
#%watermark --iversions

# Fim